In [1]:
from math import log10
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd
import os 

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader

from dataset.data_loader_RGB import *
from utils.pytorch_ssim import *

from facenet_pytorch import InceptionResnetV1
from GAN_model import Generator, Discriminator
from utils.loss import*

torch.manual_seed(1)
device = torch.device("cuda")

In [4]:
# SRGAN parameters

batch_size = 32
num_epochs = 18
lr = 1e-4
threads = 4
upscale_factor = 4

In [3]:
#img_path_low = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-56/train'
#img_path_ref = '/media/angelo/DATEN/Datasets/Experiment_Masters/300W-3D-low-res-224/train'

img_path_low = '/home/jupyter/dataset/LR_56/train/'
img_path_ref = '/home/jupyter/dataset/HR/train/'

#img_path_low = '/media/angelo/DATEN/Datasets/CelebA/LR_56/train/'
#img_path_ref = '/media/angelo/DATEN/Datasets/CelebA/HR/train/'

train_set = DatasetSuperRes(img_path_low, img_path_ref)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batch_size, shuffle=True)

In [6]:
#netG = Generator(upscale_factor).to(device)
#netD = Discriminator().to(device)

netG = torch.load('models/SRGAN_FaceLoss_x4_epoch_12.pth', map_location=lambda storage, loc: storage).to(device)
netD = torch.load('models/netD_FaceLoss_x4_epoch_12.pth', map_location=lambda storage, loc: storage).to(device)

#netG.weight_init(mean=0.0, std=0.2)
#netD.weight_init(mean=0.0, std=0.2)

feature_extraction_model = InceptionResnetV1(pretrained='vggface2').eval()

generator_criterion = GeneratorLoss().to(device)
face_loss = FaceIdentityLoss(feature_extraction_model).to(device)
MSE_criterion = nn.MSELoss()

In [7]:
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
    
results = {'d_loss': [], 'g_loss': [], 'd_score': [],
           'g_score': [], 'psnr': [], 'ssim': []}

In [8]:
# Let's pretrain the Generator
pre_train_epochs = 5
for epoch in range(1, pre_train_epochs+1):
    
    netG.train()
    
    for data, target in training_data_loader:
        data, target = data.to(device), target.to(device)
        
        netG.zero_grad()
        loss = MSE_criterion(netG(data), target)
        loss.backward()
        optimizerG.step()
    
    print('Last loss: {}'.format(loss.item()))
    print('Finished pre-training {}/{}'.format(epoch,pre_train_epochs))

Last loss: 0.0022416841238737106
Finished pre-training 1/5
Last loss: 0.0019902896601706743
Finished pre-training 2/5
Last loss: 0.0017285746289417148
Finished pre-training 3/5
Last loss: 0.00196244684047997
Finished pre-training 4/5
Last loss: 0.0018473186064511538
Finished pre-training 5/5


In [9]:
out_path = 'results/'
out_model_path = 'models/'

if not os.path.exists(out_path):
    os.makedirs(out_path)    

if not os.path.exists(out_model_path):
    os.makedirs(out_model_path)  

In [11]:
def train(epoch):
    
    running_results = {'batch_sizes': 0, 'd_loss': 0, 'g_loss': 0, 
                       'd_score': 0, 'g_score': 0}
    
    netG.train()
    netD.train()
    iteration = 0
    for data, target in training_data_loader:
        iteration+=1
        g_update_first = True
        batch_size = data.size(0)
        running_results['batch_sizes'] += batch_size

        ############################
        # (1) Update D network: maximize D(x)-1-D(G(z))
        ###########################
        real_img = Variable(target).to(device)
        z = Variable(data).to(device)
        
        fake_img = netG(z)

        netD.zero_grad()
        
        real_out = netD(real_img).mean()
        fake_out = netD(fake_img).mean()
        
        d_loss = 1 - real_out + fake_out
        
        if d_loss.item() > 0.5:
            d_loss.backward(retain_graph=True)
            optimizerD.step()

        ############################
        # (2) Update G network: minimize 1-D(G(z)) + Perception Loss + Image Loss + TV Loss + FaceID Loss
        ###########################
        netG.zero_grad()
        
        #mse_adversarial_content_loss = generator_criterion(fake_out, fake_img, real_img)
        #face_perceptual_loss = face_loss(fake_img, real_img)
        
        g_loss = generator_criterion(fake_out, fake_img, real_img) + face_loss(fake_img, real_img)
        
        g_loss.backward()

        fake_img = netG(z)
        fake_out = netD(fake_img).mean()

        optimizerG.step()

        # Loss for current batch before optimization 

        running_results['g_loss'] += g_loss.item() * batch_size
        running_results['d_loss'] += d_loss.item() * batch_size
        running_results['d_score'] += real_out.item() * batch_size
        running_results['g_score'] += fake_out.item() * batch_size

        print("===> Epoch[{}]({}/{}): D(x) {:.4f} Loss_D: {:.4f}".format(epoch, iteration, 
                                                                         len(training_data_loader), 
                                                                         real_out.item(),
                                                                         d_loss.item()))

    batch_mse = ((fake_img - real_img) ** 2).data.mean()
    batch_ssim = ssim(fake_img, real_img).item()
    batch_psnr = 10 * log10(1 /batch_mse)  
    
    print('[{}/{}] Loss_D: {:.4f} Loss_G: {:.4f} D(x): {:.4f} D(G(z)): {:.4f} PSNR {:.4f}'.format(
        epoch, num_epochs, 
        running_results['d_loss'] / running_results['batch_sizes'],
        running_results['g_loss'] / running_results['batch_sizes'],
        running_results['d_score'] / running_results['batch_sizes'],
        running_results['g_score'] / running_results['batch_sizes'], batch_psnr))
          
    # Save loss\scores\psnr\ssim
    results['d_loss'].append(running_results['d_loss'] / running_results['batch_sizes'])
    results['g_loss'].append(running_results['g_loss'] / running_results['batch_sizes'])
    results['d_score'].append(running_results['d_score'] / running_results['batch_sizes'])
    results['g_score'].append(running_results['g_score'] / running_results['batch_sizes'])
    results['psnr'].append(batch_psnr)
    results['ssim'].append(batch_ssim)

    if epoch % 2 == 0:

        # Save model parameters
        torch.save(netG, out_model_path + 'SRGAN_FaceLoss_x%d_epoch_%d.pth' % (upscale_factor, epoch+12))
        torch.save(netD, out_model_path + 'netD_FaceLoss_x%d_epoch_%d.pth' % (upscale_factor, epoch+12))

        data_frame = pd.DataFrame(
            data={'Loss_D': results['d_loss'], 'Loss_G': results['g_loss'], 'Score_D': results['d_score'],
                  'Score_G': results['g_score'], 'PSNR': results['psnr'], 'SSIM': results['ssim']},
            index=range(1, epoch + 1))
        data_frame.to_csv(out_path + 'SRGAN_FaceLoss_x' + str(upscale_factor) + '_train_results.csv', index_label='Epoch')

In [12]:
#num_epochs = 30

for epoch in range(1, 19):
    train(epoch)

===> Epoch[1](1/562): D(x) 0.9495 Loss_D: 0.0552
===> Epoch[1](2/562): D(x) 0.9768 Loss_D: 0.0301
===> Epoch[1](3/562): D(x) 0.9147 Loss_D: 0.0915
===> Epoch[1](4/562): D(x) 0.9836 Loss_D: 0.0207
===> Epoch[1](5/562): D(x) 0.9791 Loss_D: 0.0255
===> Epoch[1](6/562): D(x) 0.9811 Loss_D: 0.0225
===> Epoch[1](7/562): D(x) 0.9845 Loss_D: 0.0197
===> Epoch[1](8/562): D(x) 0.9819 Loss_D: 0.0260
===> Epoch[1](9/562): D(x) 0.9584 Loss_D: 0.0490
===> Epoch[1](10/562): D(x) 0.9835 Loss_D: 0.0222
===> Epoch[1](11/562): D(x) 0.9820 Loss_D: 0.0228
===> Epoch[1](12/562): D(x) 0.9804 Loss_D: 0.0246
===> Epoch[1](13/562): D(x) 0.9769 Loss_D: 0.0265
===> Epoch[1](14/562): D(x) 0.9839 Loss_D: 0.0212
===> Epoch[1](15/562): D(x) 0.9734 Loss_D: 0.0333
===> Epoch[1](16/562): D(x) 0.9724 Loss_D: 0.0302
===> Epoch[1](17/562): D(x) 0.9624 Loss_D: 0.0418
===> Epoch[1](18/562): D(x) 0.9402 Loss_D: 0.0627
===> Epoch[1](19/562): D(x) 0.9729 Loss_D: 0.0322
===> Epoch[1](20/562): D(x) 0.9706 Loss_D: 0.0381
===> Epoc

===> Epoch[1](164/562): D(x) 0.9660 Loss_D: 0.0389
===> Epoch[1](165/562): D(x) 0.9902 Loss_D: 0.0159
===> Epoch[1](166/562): D(x) 0.9705 Loss_D: 0.0330
===> Epoch[1](167/562): D(x) 0.9756 Loss_D: 0.0322
===> Epoch[1](168/562): D(x) 0.9611 Loss_D: 0.0579
===> Epoch[1](169/562): D(x) 0.9503 Loss_D: 0.0525
===> Epoch[1](170/562): D(x) 0.9733 Loss_D: 0.0316
===> Epoch[1](171/562): D(x) 0.9845 Loss_D: 0.0229
===> Epoch[1](172/562): D(x) 0.9743 Loss_D: 0.0305
===> Epoch[1](173/562): D(x) 0.9723 Loss_D: 0.0320
===> Epoch[1](174/562): D(x) 0.9411 Loss_D: 0.0639
===> Epoch[1](175/562): D(x) 0.9766 Loss_D: 0.0265
===> Epoch[1](176/562): D(x) 0.9848 Loss_D: 0.0319
===> Epoch[1](177/562): D(x) 0.9706 Loss_D: 0.0329
===> Epoch[1](178/562): D(x) 0.9666 Loss_D: 0.0378
===> Epoch[1](179/562): D(x) 0.9497 Loss_D: 0.0547
===> Epoch[1](180/562): D(x) 0.9697 Loss_D: 0.0386
===> Epoch[1](181/562): D(x) 0.9461 Loss_D: 0.0601
===> Epoch[1](182/562): D(x) 0.9750 Loss_D: 0.0309
===> Epoch[1](183/562): D(x) 0.

===> Epoch[1](325/562): D(x) 0.9585 Loss_D: 0.0464
===> Epoch[1](326/562): D(x) 0.9352 Loss_D: 0.0715
===> Epoch[1](327/562): D(x) 0.9873 Loss_D: 0.0278
===> Epoch[1](328/562): D(x) 0.9415 Loss_D: 0.0652
===> Epoch[1](329/562): D(x) 0.9859 Loss_D: 0.0202
===> Epoch[1](330/562): D(x) 0.9812 Loss_D: 0.0290
===> Epoch[1](331/562): D(x) 0.9790 Loss_D: 0.0257
===> Epoch[1](332/562): D(x) 0.9613 Loss_D: 0.0466
===> Epoch[1](333/562): D(x) 0.9735 Loss_D: 0.0322
===> Epoch[1](334/562): D(x) 0.9509 Loss_D: 0.0542
===> Epoch[1](335/562): D(x) 0.9571 Loss_D: 0.0512
===> Epoch[1](336/562): D(x) 0.9710 Loss_D: 0.0368
===> Epoch[1](337/562): D(x) 0.9692 Loss_D: 0.0375
===> Epoch[1](338/562): D(x) 0.9520 Loss_D: 0.0650
===> Epoch[1](339/562): D(x) 0.9675 Loss_D: 0.0379
===> Epoch[1](340/562): D(x) 0.9355 Loss_D: 0.0679
===> Epoch[1](341/562): D(x) 0.9696 Loss_D: 0.0357
===> Epoch[1](342/562): D(x) 0.9293 Loss_D: 0.0753
===> Epoch[1](343/562): D(x) 0.9730 Loss_D: 0.0310
===> Epoch[1](344/562): D(x) 0.

===> Epoch[1](486/562): D(x) 0.9596 Loss_D: 0.0491
===> Epoch[1](487/562): D(x) 0.9798 Loss_D: 0.0270
===> Epoch[1](488/562): D(x) 0.9891 Loss_D: 0.0156
===> Epoch[1](489/562): D(x) 0.9825 Loss_D: 0.0223
===> Epoch[1](490/562): D(x) 0.9765 Loss_D: 0.0280
===> Epoch[1](491/562): D(x) 0.9583 Loss_D: 0.0478
===> Epoch[1](492/562): D(x) 0.9466 Loss_D: 0.0596
===> Epoch[1](493/562): D(x) 0.9044 Loss_D: 0.1043
===> Epoch[1](494/562): D(x) 0.9280 Loss_D: 0.0771
===> Epoch[1](495/562): D(x) 0.9875 Loss_D: 0.0164
===> Epoch[1](496/562): D(x) 0.9674 Loss_D: 0.0396
===> Epoch[1](497/562): D(x) 0.9663 Loss_D: 0.0381
===> Epoch[1](498/562): D(x) 0.9785 Loss_D: 0.0280
===> Epoch[1](499/562): D(x) 0.9680 Loss_D: 0.0380
===> Epoch[1](500/562): D(x) 0.9755 Loss_D: 0.0275
===> Epoch[1](501/562): D(x) 0.9811 Loss_D: 0.0228
===> Epoch[1](502/562): D(x) 0.9833 Loss_D: 0.0241
===> Epoch[1](503/562): D(x) 0.9798 Loss_D: 0.0309
===> Epoch[1](504/562): D(x) 0.9829 Loss_D: 0.0246
===> Epoch[1](505/562): D(x) 0.

===> Epoch[2](85/562): D(x) 0.9437 Loss_D: 0.0626
===> Epoch[2](86/562): D(x) 0.9262 Loss_D: 0.0800
===> Epoch[2](87/562): D(x) 0.9554 Loss_D: 0.0505
===> Epoch[2](88/562): D(x) 0.9452 Loss_D: 0.0606
===> Epoch[2](89/562): D(x) 0.9709 Loss_D: 0.0348
===> Epoch[2](90/562): D(x) 0.9789 Loss_D: 0.0258
===> Epoch[2](91/562): D(x) 0.9789 Loss_D: 0.0352
===> Epoch[2](92/562): D(x) 0.9686 Loss_D: 0.0395
===> Epoch[2](93/562): D(x) 0.9469 Loss_D: 0.0631
===> Epoch[2](94/562): D(x) 0.9653 Loss_D: 0.0391
===> Epoch[2](95/562): D(x) 0.9760 Loss_D: 0.0290
===> Epoch[2](96/562): D(x) 0.9852 Loss_D: 0.0199
===> Epoch[2](97/562): D(x) 0.9773 Loss_D: 0.0285
===> Epoch[2](98/562): D(x) 0.9784 Loss_D: 0.0270
===> Epoch[2](99/562): D(x) 0.9877 Loss_D: 0.0214
===> Epoch[2](100/562): D(x) 0.9802 Loss_D: 0.0248
===> Epoch[2](101/562): D(x) 0.9685 Loss_D: 0.0380
===> Epoch[2](102/562): D(x) 0.9757 Loss_D: 0.0304
===> Epoch[2](103/562): D(x) 0.9388 Loss_D: 0.0654
===> Epoch[2](104/562): D(x) 0.9605 Loss_D: 0.

...

===> Epoch[17](440/562): D(x) 0.9515 Loss_D: 0.0546
===> Epoch[17](441/562): D(x) 0.9615 Loss_D: 0.0437
===> Epoch[17](442/562): D(x) 0.9734 Loss_D: 0.0329
===> Epoch[17](443/562): D(x) 0.9431 Loss_D: 0.0627
===> Epoch[17](444/562): D(x) 0.9474 Loss_D: 0.0565
===> Epoch[17](445/562): D(x) 0.9736 Loss_D: 0.0302
===> Epoch[17](446/562): D(x) 0.9672 Loss_D: 0.0373
===> Epoch[17](447/562): D(x) 0.9801 Loss_D: 0.0245
===> Epoch[17](448/562): D(x) 0.9527 Loss_D: 0.0526
===> Epoch[17](449/562): D(x) 0.9767 Loss_D: 0.0273
===> Epoch[17](450/562): D(x) 0.9775 Loss_D: 0.0265
===> Epoch[17](451/562): D(x) 0.9374 Loss_D: 0.0682
===> Epoch[17](452/562): D(x) 0.9671 Loss_D: 0.0362
===> Epoch[17](453/562): D(x) 0.9578 Loss_D: 0.0475
===> Epoch[17](454/562): D(x) 0.9374 Loss_D: 0.0681
===> Epoch[17](455/562): D(x) 0.9460 Loss_D: 0.0592
===> Epoch[17](456/562): D(x) 0.9829 Loss_D: 0.0239
===> Epoch[17](457/562): D(x) 0.9605 Loss_D: 0.0452
===> Epoch[17](458/562): D(x) 0.9646 Loss_D: 0.0472
===> Epoch[1

===> Epoch[18](35/562): D(x) 0.9825 Loss_D: 0.0216
===> Epoch[18](36/562): D(x) 0.9659 Loss_D: 0.0391
===> Epoch[18](37/562): D(x) 0.9762 Loss_D: 0.0270
===> Epoch[18](38/562): D(x) 0.9304 Loss_D: 0.0758
===> Epoch[18](39/562): D(x) 0.9650 Loss_D: 0.0383
===> Epoch[18](40/562): D(x) 0.9851 Loss_D: 0.0225
===> Epoch[18](41/562): D(x) 0.9739 Loss_D: 0.0313
===> Epoch[18](42/562): D(x) 0.9431 Loss_D: 0.0633
===> Epoch[18](43/562): D(x) 0.9856 Loss_D: 0.0181
===> Epoch[18](44/562): D(x) 0.9758 Loss_D: 0.0323
===> Epoch[18](45/562): D(x) 0.9574 Loss_D: 0.0482
===> Epoch[18](46/562): D(x) 0.9706 Loss_D: 0.0340
===> Epoch[18](47/562): D(x) 0.9626 Loss_D: 0.0423
===> Epoch[18](48/562): D(x) 0.9749 Loss_D: 0.0330
===> Epoch[18](49/562): D(x) 0.9697 Loss_D: 0.0335
===> Epoch[18](50/562): D(x) 0.9859 Loss_D: 0.0201
===> Epoch[18](51/562): D(x) 0.9822 Loss_D: 0.0228
===> Epoch[18](52/562): D(x) 0.9611 Loss_D: 0.0437
===> Epoch[18](53/562): D(x) 0.9624 Loss_D: 0.0418
===> Epoch[18](54/562): D(x) 0.

===> Epoch[18](194/562): D(x) 0.9635 Loss_D: 0.0430
===> Epoch[18](195/562): D(x) 0.9639 Loss_D: 0.0408
===> Epoch[18](196/562): D(x) 0.9814 Loss_D: 0.0242
===> Epoch[18](197/562): D(x) 0.9840 Loss_D: 0.0215
===> Epoch[18](198/562): D(x) 0.9803 Loss_D: 0.0236
===> Epoch[18](199/562): D(x) 0.9845 Loss_D: 0.0199
===> Epoch[18](200/562): D(x) 0.9759 Loss_D: 0.0317
===> Epoch[18](201/562): D(x) 0.9841 Loss_D: 0.0208
===> Epoch[18](202/562): D(x) 0.9708 Loss_D: 0.0339
===> Epoch[18](203/562): D(x) 0.9830 Loss_D: 0.0231
===> Epoch[18](204/562): D(x) 0.9679 Loss_D: 0.0400
===> Epoch[18](205/562): D(x) 0.9721 Loss_D: 0.0329
===> Epoch[18](206/562): D(x) 0.9902 Loss_D: 0.0145
===> Epoch[18](207/562): D(x) 0.9702 Loss_D: 0.0355
===> Epoch[18](208/562): D(x) 0.9367 Loss_D: 0.0696
===> Epoch[18](209/562): D(x) 0.9669 Loss_D: 0.0382
===> Epoch[18](210/562): D(x) 0.9582 Loss_D: 0.0468
===> Epoch[18](211/562): D(x) 0.9652 Loss_D: 0.0402
===> Epoch[18](212/562): D(x) 0.9806 Loss_D: 0.0343
===> Epoch[1

===> Epoch[18](352/562): D(x) 0.9733 Loss_D: 0.0311
===> Epoch[18](353/562): D(x) 0.8899 Loss_D: 0.1170
===> Epoch[18](354/562): D(x) 0.9870 Loss_D: 0.0170
===> Epoch[18](355/562): D(x) 0.9804 Loss_D: 0.0239
===> Epoch[18](356/562): D(x) 0.9410 Loss_D: 0.0649
===> Epoch[18](357/562): D(x) 0.9482 Loss_D: 0.0571
===> Epoch[18](358/562): D(x) 0.9842 Loss_D: 0.0210
===> Epoch[18](359/562): D(x) 0.9793 Loss_D: 0.0266
===> Epoch[18](360/562): D(x) 0.9425 Loss_D: 0.0614
===> Epoch[18](361/562): D(x) 0.9499 Loss_D: 0.0547
===> Epoch[18](362/562): D(x) 0.9428 Loss_D: 0.0636
===> Epoch[18](363/562): D(x) 0.9401 Loss_D: 0.0663
===> Epoch[18](364/562): D(x) 0.8998 Loss_D: 0.1044
===> Epoch[18](365/562): D(x) 0.9689 Loss_D: 0.0358
===> Epoch[18](366/562): D(x) 0.9600 Loss_D: 0.0446
===> Epoch[18](367/562): D(x) 0.9802 Loss_D: 0.0254
===> Epoch[18](368/562): D(x) 0.9817 Loss_D: 0.0253
===> Epoch[18](369/562): D(x) 0.9618 Loss_D: 0.0432
===> Epoch[18](370/562): D(x) 0.9721 Loss_D: 0.0335
===> Epoch[1

===> Epoch[18](510/562): D(x) 0.9935 Loss_D: 0.0129
===> Epoch[18](511/562): D(x) 0.9531 Loss_D: 0.0521
===> Epoch[18](512/562): D(x) 0.9892 Loss_D: 0.0162
===> Epoch[18](513/562): D(x) 0.9905 Loss_D: 0.0154
===> Epoch[18](514/562): D(x) 0.9923 Loss_D: 0.0126
===> Epoch[18](515/562): D(x) 0.9776 Loss_D: 0.0335
===> Epoch[18](516/562): D(x) 0.9777 Loss_D: 0.0317
===> Epoch[18](517/562): D(x) 0.9683 Loss_D: 0.0373
===> Epoch[18](518/562): D(x) 0.9492 Loss_D: 0.0580
===> Epoch[18](519/562): D(x) 0.9609 Loss_D: 0.0452
===> Epoch[18](520/562): D(x) 0.8933 Loss_D: 0.1129
===> Epoch[18](521/562): D(x) 0.9638 Loss_D: 0.0411
===> Epoch[18](522/562): D(x) 0.9667 Loss_D: 0.0385
===> Epoch[18](523/562): D(x) 0.9572 Loss_D: 0.0471
===> Epoch[18](524/562): D(x) 0.9888 Loss_D: 0.0155
===> Epoch[18](525/562): D(x) 0.9691 Loss_D: 0.0371
===> Epoch[18](526/562): D(x) 0.9486 Loss_D: 0.0581
===> Epoch[18](527/562): D(x) 0.9734 Loss_D: 0.0331
===> Epoch[18](528/562): D(x) 0.9667 Loss_D: 0.0397
===> Epoch[1